In [1]:
from multiprocessing import Pool
from pystac_client import Client  
# from collections import defaultdict    
# import json
# import geopandas
# from cartopy import crs
import geopandas as gpd
# import glob
# import netCDF4
from datetime import datetime
import os
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, shape
from itertools import repeat
import netCDF4 as ncf
from itertools import chain
import datetime
from datetime import datetime, timedelta
import json


# # Local importse
# from datagen.S3List import S3List


def generate_time_search(timekey):
        # timekey = "2024-01-01T00:00:00Z,2024-04-01T23:59:59Z"
        time1 = timekey.split(',')[0].split('T')[0]
        all_date = [int(i) for i in time1.split('-')]
        time2 = timekey.split(',')[1].split('T')[0]
        final_hours = [i for i in timekey.split(',')[1].split('T')[1].split(':')]
        all_date2 = [int(i) for i in time2.split('-')]


        start_date = datetime(all_date[0], all_date[1], all_date[2])
        end_date = datetime(all_date2[0], all_date2[1], all_date2[2])

        add_days = timedelta(days=30)
        add_ending_hours = timedelta(hours = int(final_hours[0]), minutes=int(final_hours[1]), seconds=int(final_hours[2][:-1]))


        start_dates = []
        ending_dates = []

        while start_date <= end_date:
            start_dates.append(start_date)
            start_date += add_days
            ending_dates.append(start_date)

        ending_dates[-1] = end_date + add_ending_hours

        parsed_dates = []

        for i in range(len(start_dates)):
            
            
            parsed_dates.append(','.join([start_dates[i].strftime('%Y-%m-%dT%H:%M:%SZ'), ending_dates[i].strftime('%Y-%m-%dT%H:%M:%SZ')]))
        return parsed_dates


# find hls tiles given a point

def find_hls_tiles(line_geo=False, band=False, limit=False, collections = ['HLSL30.v2.0', 'HLSS30.v2.0'], date_range = False):

    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


    catalog = Client.open(f'{STAC_URL}/LPCLOUD/')



    # try:
    #     x, y = point[0], point[1]
    #     # print(x,y)
    # except TypeError:
    #     print("Point must be in the form of [lat,lon]")
    #     raise

    # point = geopandas.points_from_xy([x],[y])
    # point = point[0]

    # date_range = '2017-01-01T00:00:00Z/2018-12-31T23:59:59Z'
    if date_range == False:
# ['2020-01-01:00:00:00Z', '..']
        # search = catalog.search(
        #     collections=collections, intersects = line_geo, datetime=date_range.replace(',', '/'))
        raise ValueError('Please supply a date for ssc...')
    else:
        all_temporal_ranges = generate_time_search(date_range)
        links = []
        for i in all_temporal_ranges:
            search = catalog.search(
                collections=collections, intersects = line_geo, datetime=i.replace(',', '/'))


            # print(f'{search.matched()} Tiles Found...')


            item_collection = search.get_all_items()

            if limit:
                item_collection = item_collection[:limit]

            if band:
                if type(band) == list:
                    for i in item_collection:
                        for b in band:
                            link = i.assets[b].href
                            # print(link)
                            links.append(link)
                
                else:
                    for i in item_collection:
                        link = i.assets[band].href
                        links.append(link)
            
            else:
                for i in item_collection:
                    # print(i.assets)
                    for key in i.assets:
                        if key.startswith('B'):
                            # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
                            link = i.assets[key].href

                            # print(link)
                            links.append(link)

        return links

def find_download_links_for_reach_tiles(data_dir, reach_id, cont, temporal_range):
    try:
        lat_list, lon_list = get_reach_node_cords(data_dir,reach_id, cont)
        

        df = pd.DataFrame(columns=['x', 'y'])
        df['x'] = lat_list[:5]
        df['y'] = lon_list[:5]
        df['ID'] = reach_id
        geometry = [Point(xy) for xy in zip(df.x, df.y)]



        geo_df = gpd.GeoDataFrame(df, geometry=geometry)

        geo_df2 = geo_df.groupby(['ID'])['geometry'].apply(lambda x: LineString(x.tolist()))
        geo_df2 = gpd.GeoDataFrame(geo_df2, geometry='geometry')
        line_geo = list(geo_df2.geometry.unique())[0]
        links = find_hls_tiles(line_geo=line_geo, date_range=temporal_range)

        outpath = '/storage/data/ssc_matchup/'

        with open(os.path.join(outpath,f'{reach_id}.json'), 'w') as f:
            json.dump(links, f)
            link = []



    except Exception as e:
        links = ['foo']
        print('error on ', reach_id)
        print(e)

    return list(set(links))





def get_reach_node_cords(sword_path, reach_id, cont):

    lat_list, lon_list = [], []

    # sword_fp = os.path.join(data_dir, f'{cont.lower()}_sword_v15.nc')
    # print(f'Searching across {len(files)} continents for nodes...')


    rootgrp = ncf.Dataset(sword_path, "r", format="NETCDF4")

    node_ids_indexes = np.where(rootgrp.groups['nodes'].variables['reach_id'][:].data.astype('U') == str(reach_id))

    if len(node_ids_indexes[0])!=0:
        for y in node_ids_indexes[0]:

            lat = float(rootgrp.groups['nodes'].variables['x'][y].data.astype('U'))
            lon = float(rootgrp.groups['nodes'].variables['y'][y].data.astype('U'))
            # all_nodes.append([lat,lon])
            lat_list.append(lat)
            lon_list.append(lon)


        rootgrp.close()

    # print(f'Found {len(all_nodes)} nodes...')
    return lat_list, lon_list












# all_links = find_download_links_for_reach_tiles('/home/confluence/data/mnt/input/sword',23216000521)
# print(len(all_links))



def ssc_process_continent(reach_ids, cont, data_dir, temporal_range):


    pool = Pool(processes=7)              # start 4 worker processes
    result = pool.starmap(find_download_links_for_reach_tiles, zip(repeat(data_dir), reach_ids, repeat(cont), repeat(temporal_range)))
    # cnt = 0
    # for i in result:
    #     for x in i:
    #         cnt += 1

    pool.close()

    flatten_list = list(chain.from_iterable(result))
    flatten_list = list(set(flatten_list))
    no_bands = list(set([i[:-10] for i in flatten_list]))
    print(f'Found {len(no_bands)} scenes for {cont}...')
    return no_bands




 

In [2]:
import os
import json
import warnings
warnings.filterwarnings('ignore')
temporal_range = "2023-01-01T00:00:00Z,2024-04-25T23:59:59Z"
cont = 'NA'
data_dir = '/storage/data/hpc/mnt/input/sword/na_sword_v16.nc'

with open(os.path.join('/storage/data/hpc/mnt/', 'input', 'reaches.json')) as f:
    reach_ids = json.load(f)

reach_ids = [i['reach_id'] for i in reach_ids]


no_bands = ssc_process_continent(reach_ids, cont, data_dir, temporal_range)

error on  74262200221
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262200221.json/w'
error on  74262300371
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262300371.json/w'
error on  74262400111
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262400111.json/w'
error on  74262600213
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262600213.json/w'
error on  74262700303
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262700303.json/w'
error on  74262300864
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262300864.json/w'
error on  74262200231
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262200231.json/w'
error on  74262300383
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262300383.json/w'
error on  74262400121
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262400121.json/w'
error on  74262600223
[Errno 2] No such file o

KeyboardInterrupt: 

error on  74262200301
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262200301.json/w'
error on  74262400183
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262400183.json/w'
error on  74261000074
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74261000074.json/w'
error on  74262300443
[Errno 2] No such file or directory: '/storage/data/ssc_matchup/74262300443.json/w'


In [ ]:
no_bands

In [ ]:
# Travis -  run hls matchup and give you a list of tiles
# Travis - list of reach IDs and dates you have discharge
# Luisa - check for overlap between days we have discharge for a reach and the hls tiles
# Luisa - list of reaches and tiles that overlap
# Travis - preprocess them
# Luisa - runs ANN, analyses, and so on